In [2]:
!pip install gradio plotly --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


In [3]:
import gradio as gr
import requests
import pandas as pd
import plotly.express as px

def get_crypto_price(crypto_ids):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={','.join(crypto_ids)}&vs_currencies=usd"
    response = requests.get(url).json()
    return {crypto: response[crypto]['usd'] for crypto in crypto_ids if crypto in response}

def calculate_portfolio(bitcoin, ethereum, dogecoin, solana, cardano):
    crypto_options = ["bitcoin", "ethereum", "dogecoin", "solana", "cardano"]
    portfolio = {"bitcoin": bitcoin, "ethereum": ethereum, "dogecoin": dogecoin, "solana": solana, "cardano": cardano}

    crypto_prices = get_crypto_price(crypto_options)
    total_value = sum(portfolio[crypto] * crypto_prices.get(crypto, 0) for crypto in portfolio)

    # Create DataFrame for visualization
    data = {"Crypto": [], "Holdings (USD)": []}
    for crypto, amount in portfolio.items():
        if amount > 0 and crypto in crypto_prices:
            data["Crypto"].append(crypto.capitalize())
            data["Holdings (USD)"].append(amount * crypto_prices[crypto])

    df = pd.DataFrame(data)

    # Portfolio allocation chart
    if not df.empty:
        fig = px.pie(df, names="Crypto", values="Holdings (USD)", title="Portfolio Allocation", hole=0.4)
        return f"Portfolio Value: ${total_value:,.2f}", df, fig
    else:
        return f"Portfolio Value: ${total_value:,.2f}", df, None

iface = gr.Interface(
    fn=calculate_portfolio,
    inputs=[
        gr.Number(label="Bitcoin (Amount Held)", value=0.0),
        gr.Number(label="Ethereum (Amount Held)", value=0.0),
        gr.Number(label="Dogecoin (Amount Held)", value=0.0),
        gr.Number(label="Solana (Amount Held)", value=0.0),
        gr.Number(label="Cardano (Amount Held)", value=0.0),
    ],
    outputs=["text", "dataframe", gr.Plot()],
    title="💰 Simple Crypto Portfolio Tracker",
    description="Track your cryptocurrency holdings in real-time!",
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a63aecf7baa3222a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
